# Gender Difference: XGB Experiment based on 59 Variables 1513k Rows 14 Waves

In [1]:
%pwd

'D:\\OneDrive - Kyushu University\\ESG09_Article\\Code'

In [2]:
%cd ..

D:\OneDrive - Kyushu University\ESG09_Article


## Import Packagee

In [3]:
import os 
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import xgboost as xgb

## Load and Make Datasets

In [4]:
Df_Filename = os.path.join("Data", "GallupWB_Ml60var1513k14wave_v1.parquet")

In [5]:
Df = pd.read_parquet(Df_Filename)

In [6]:
Df.shape

(1513459, 59)

In [7]:
Df.columns

Index(['wave', 'INCOME_2', 'Cantril_ladder', 'Health_disable', 'Relative_have',
       'Living_standard_change', 'Enough_food', 'Enough_shelter',
       'Well_rested', 'Respected', 'Smile', 'Interesting_thing', 'Enjoyment',
       'Physical_pain', 'Worry', 'Sadness', 'Stress', 'Anger',
       'Economic_change', 'Goodtime_job', 'Sat_pubtran', 'Sat_road', 'Sat_edu',
       'Sat_qualityair', 'Sat_qualitywater', 'Sat_healthcare', 'Sat_affhouse',
       'Sat_oppofriend', 'Good_minorities', 'Good_gayles', 'Good_immigrants',
       'Donated', 'Volunteer', 'Help_stranger', 'Voice_official',
       'Local_police', 'Safety_walk', 'Stolen', 'Assualted',
       'Religion_importance', 'Children_respected', 'Children_learn',
       'Women_respected', 'Conf_military', 'Conf_judicial', 'Conf_government',
       'Conf_financial', 'Conf_honestyelections', 'Freedom_media',
       'Corruption_business', 'Corruption_government',
       'Performance_leadership', 'Age', 'Marital_status', 'Employment',
      

In [9]:
female_cantrilladder = Df.loc[Df['Gender_female']==2, 'Cantril_ladder'].mean()

KeyError: 'Gender_female'